In [1]:
from keras.preprocessing.image import img_to_array, load_img

target_img_path = '...'
style_reference_image_path = '...'

width, height = load_img(target_img_path).size
img_height = 400
img_width = 400

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    x[:,:,0] += 103.939
    x[:,:,1] += 116.779
    x[:,:,2] += 123.68
    x = x[:,:,::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [3]:
from keras import backend as K

target_image = K.constant(preprocess_image(target_img_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor, weights=None, include_top=False)
model.load_weights('C:/Users/Ahmed/.keras/models/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')

print('Model loaded.')

Model loaded.


In [4]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

def gram_mat(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_mat(style)
    C = gram_mat(combination)
    channels = 3
    size = img_height*img_width
    return K.sum(K.square(S - C) / (4.* (size**2) * (channels**2)))

def total_variation_loss(x):
    a = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, 1:, :img_width-1, :])
    b = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, :img_height-1, 1:, :])
    return K.sum(K.pow(a+b, 1.25))

In [5]:
output_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

In [6]:
loss = K.variable(0.)
layer_features = output_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight*content_loss(target_image_features, combination_features)

for layer_name in style_layers:
    layer_features = output_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight/len(style_layers))*sl
    
loss += total_variation_weight*total_variation_loss(combination_image)

In [7]:
grads = K.gradients(loss, combination_image)[0]

fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grad_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape([1, img_height, img_width, 3])
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values
    
evaluator = Evaluator()

#  Adding Style to the Content Image

In [8]:
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

result_prefix = 'my_result'
iterations = 20

# x = preprocess_image(target_img_path)
x = black_image()
x = x.flatten()

for i in range(iterations):
    print('Start of iteration', i+1)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iter_%d.png' %(i+1)
    imsave(fname, img)
    print('Image saved as '+fname)
    end_time = time.time()
    print('Iteration %d completed in %ds'%(i+1, end_time-start_time))

Start of iteration 1
Current loss value: 28345752000.0
Image saved as my_result_at_iter_1.png
Iteration 1 completed in 584s
Start of iteration 2


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Current loss value: 6560500000.0
Image saved as my_result_at_iter_2.png
Iteration 2 completed in 597s
Start of iteration 3
Current loss value: 4029869800.0
Image saved as my_result_at_iter_3.png
Iteration 3 completed in 596s
Start of iteration 4
Current loss value: 2894352600.0
Image saved as my_result_at_iter_4.png
Iteration 4 completed in 594s
Start of iteration 5
Current loss value: 2306539800.0
Image saved as my_result_at_iter_5.png
Iteration 5 completed in 592s
Start of iteration 6
Current loss value: 1948662100.0
Image saved as my_result_at_iter_6.png
Iteration 6 completed in 593s
Start of iteration 7
Current loss value: 1688664200.0
Image saved as my_result_at_iter_7.png
Iteration 7 completed in 577s
Start of iteration 8
Current loss value: 1515555600.0
Image saved as my_result_at_iter_8.png
Iteration 8 completed in 579s
Start of iteration 9
Current loss value: 1370147000.0
Image saved as my_result_at_iter_9.png
Iteration 9 completed in 586s
Start of iteration 10
Current loss va